# 🧠 Project Overview: AI Research Assistant

## Problem Statement:
Reading, understanding, and comparing academic papers is time-consuming for students and researchers. Often, papers come in PDF format, have dense language, and lack tools for comparison across related work.

🧠 Proposed Solution with GenAI:
We build an AI-powered academic assistant that:

- Extracts and summarizes research papers (PDF, URL, or plain text)

- Retrieves topically similar papers from arXiv

- Uses vector embeddings + FAISS for similarity ranking

- Compares the user’s paper with similar papers

- Generates structured summaries, outlines, and feedback

This project demonstrates **6 GenAI** capabilities:

| Capability                               | How it's Used                                                                                                                                                                      |
| ---------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| ✅Document Understanding                  | PDF or URL-based papers are available to be parsed and can be summarized using Gemini if the user includes with the input method. Various input methods for users were implicated. |
| ✅ Few-shot Prompting                     | Multiple few-shot prompts used for summarization, comparison, and writing evaluation.                                                                                              |
| ✅ Long Context Window                    | Gemini processes full-text research papers (from PDF or text) — this qualifies.                                                                                                    |
| ✅ GenAI Evaluation                       | Gemini evaluates and gives feedback on a student’s academic writing sample.                                                                                                        |
| ✅ Embeddings / Vector Search / Vector DB | SentenceTransformers + FAISS are used for similarity search based on research topic.                                                                                               |
| ✅ RAG                                    | arXiv retrieval + Gemini generation to compare user paper with others.        

 # Disclaimer ⚠️  
 As engineers of AI we are responsible the for moral and ethical concerns surrounding it's development. In this project, our aims are only to streamline the research process for all researchers so that they can focus on the content of their research with ease. You must note the following when using such a product: 

- Large Language Models are still prone to AI hallucination regardless of the sophistication of the model. While in our experience the model has not done so, please read all the output carefully before forming arguments surrounding them.
- Ensure that any files you use are files you have the right to use, corresponding to the copyright laws subject to where you are located.
- Please be mindful of the methods in which you use this Agent: Do not utilise it to find specific academic arguments out of context. That is dangerous, misleading and not in the spirit of good academic research.

We do not condone the use of AI to author research or that researchers take credit for AI generated content used with or without modification in the resulting academia. We do not recommend replacing your academic sense with this agent either, user discretion is advised. We are in no way responsible for irregular or malicious use of this agent.

## 1. Authentications and Imports

In [1]:
!pip install -q google-generativeai sentence-transformers faiss-cpu PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 45.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.0 MB/s eta 0:00:00


In [2]:

import os
import google.generativeai as genai
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

!pip uninstall -qy jupyterlab 
!pip install -U -q "google-genai==1.7.0"

from google import genai
from google.genai import types
from google.api_core import retry
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient

# Adding the automatic retry codes for transient errors
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
genai.models.Models.generate_content = retry.Retry(predicate=is_retriable)(genai.models.Models.generate_content)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.6 MB/s eta 0:00:00


In [3]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 73.9 MB/s eta 0:00:00:00:0100:01


### 1.2 Adding the GOOGLE API KEY

In [4]:
# Load your API key from Kaggle Add-ons as a secret
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

## 📥 2. Input Modes: PDF / Text / URL

#### We allow 3 flexible input types for papers:
 - Upload local PDF
 - Paste full text manually
 - Fetch from URL (e.g., arXiv)

##### So, in the cell below, Upload PDF or Url or use Text

In [5]:
import fitz  # PyMuPDF
import requests
import os

def extract_text_with_fitz(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_text_from_pdf_url(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open("downloaded_paper.pdf", 'wb') as f:
                f.write(response.content)
            return extract_text_with_fitz("downloaded_paper.pdf")
        else:
            raise Exception("⚠️ Failed to fetch paper (not 200 OK).")
    except Exception as e:
        raise Exception(
            f"❌ Could not fetch the PDF from the internet.\n"
            f"👉 This is expected in Kaggle — please upload the PDF manually.\nError: {str(e)}"
        )

# Set mode: 'pdf', 'text', 'url'
input_mode = 'pdf'  # 'pdf', 'text', or 'url'

if input_mode == 'pdf':
    pdf_path = "/kaggle/input/sample-data/sample_paper.pdf"  # Update this if needed
    if os.path.exists(pdf_path):
        paper_text = extract_text_with_fitz(pdf_path)
    else:
        raise FileNotFoundError("❌ PDF not found at the specified path.")

elif input_mode == 'text':
     #Your full paper text, abstract, or sample paragraph goes here.
    paper_text =  """
Title: The Effects of AI Tutors on Student Engagement in Online Learning
Author: Eda Yılmaz
Institution: Department of Educational Technology, Istanbul University
Date: March 2024

Abstract:
This study investigates the potential of AI-powered chatbots to improve student engagement in online learning environments. While AI tutors are widely used in tech support, their role in education is still evolving. This paper presents early findings from an experimental setup where students interacted with a simple AI assistant during their online coursework.

Introduction:
The rise of online learning platforms has created both new opportunities and challenges in student engagement. AI chatbots, capable of providing instant feedback and support, might help address issues such as lack of motivation and low participation. However, limited empirical research has been done on their effectiveness in academic contexts.

Methodology:
We conducted a preliminary trial with 30 undergraduate students enrolled in a blended learning course. A rule-based AI tutor was integrated into the LMS, answering content-related questions and providing encouragement during assignments.

Results:
Initial data shows an increase in the number of voluntary questions submitted by students. However, the impact on performance is inconclusive. Some students reported confusion due to the chatbot's limitations in answering complex questions.

(Work in Progress – Conclusion and final analysis pending. Interviews and final grades will be added after April 15.)
"""


elif input_mode == 'url':
    paper_url = "https://arxiv.org/pdf/2504.11741"
    try:
        paper_text = extract_text_from_pdf_url(paper_url)
    except Exception as e:
        print("⚠️ Could not fetch paper from URL. Please upload the PDF manually.")
        paper_text = "The paper could not be loaded. Please try another method."

else:
    raise ValueError("Invalid input mode. Use 'pdf', 'text', or 'url'.")

# Preview the extracted content
print("📄 Extracted topic text:\n", paper_text[:1000])




📄 Extracted topic text:
 The EUROCALL Review, Volume 25, No. 2, September 2017 
 
18 
Research paper 
 
A look at advanced learners’ use of mobile devices for 
English language study: Insights from interview data 
Mariusz Kruk 
University of Zielona Gora, Poland 
______________________________________________________________ 
mkruk @ uz.zgora.pl 
  
Abstract 
The paper discusses the results of a study which explored advanced learners of English 
engagement with their mobile devices to develop learning experiences that meet their 
needs and goals as foreign language learners. The data were collected from 20 students 
by means of a semi-structured interview. The gathered data were subjected to 
qualitative and quantitative analysis. The results of the study demonstrated that, on the 
one hand, some subjects manifested heightened awareness relating to the 
advantageous role of mobile devices in their learning endeavors, their ability to reach 
for suitable tools and retrieve necessary inf

## 📥 3. Summarizing the Paper with Gemini
- Use Gemini with few-shot prompt

Output includes: Title, Focus, Methods, Findings, Limitations
- Use Gemini with few-shot prompt

Output includes: Title, Focus, Methods, Findings, Limitations

### 3.1

In [6]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"Summarize this academic paper:\n\n{paper_text}"
)

summary = response.text
print("📌 Summary:\n", summary)


📌 Summary:
 This paper by Mariusz Kruk investigates how advanced English language learners utilize mobile devices for autonomous language study.  The study involved semi-structured interviews with 20 Polish university students of English philology.  The data, analyzed both qualitatively and quantitatively, revealed that students generally recognized the benefits of mobile devices for language learning, particularly for vocabulary acquisition, due to convenience and easy access to resources.  However, the study also found a tendency towards intuitive use and a lack of engagement with other language skills and subsystems.  The author concludes that language teachers should actively incorporate mobile technology into their teaching to promote more effective and independent language learning by showing the affordances of mobile technology and discussing them with students during language classes, selecting appropriate mobile apps and offering or designing tasks devoted to practicing a vari

### 3.2 Detailing the Paper's Content with Few-Shot Prompting


In [7]:
# FEW-SHOT SUMMARIZATION PROMPT
few_shot_summary_prompt = f"""
You are an academic summarization assistant. Given an academic paper, write a clear and structured summary including:
- Title
- Main focus
- Methodology (if mentioned)
- Key findings
- Open questions or limitations (if available)

Example:

Paper:
"Title: The Role of AI in Personalized Learning Environments
Abstract: This paper explores the role of artificial intelligence in tailoring educational content to individual learners. A pilot study with 40 students shows that personalized feedback from AI tutors improved engagement and retention by 20%. The system used rule-based responses and tracked progress across assignments. However, limitations included lack of adaptability to open-ended queries."

Summary:
- **Title**: The Role of AI in Personalized Learning Environments
- **Focus**: How AI improves personalization in education
- **Methodology**: Pilot study with 40 students using AI tutors
- **Findings**: Improved engagement and retention by 20%
- **Limitations**: Rule-based system struggled with complex questions

Now summarize the following paper:

{paper_text}
"""

#The Gemini Model
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=few_shot_summary_prompt
)

#Output
summary = response.text
print("📌 Summary:\n", summary)


📌 Summary:
 - **Title**: A look at advanced learners’ use of mobile devices for English language study: Insights from interview data
- **Focus**: How advanced English learners use mobile devices for language study and how it meets their needs and goals
- **Methodology**: Semi-structured interviews with 20 Polish university students of English philology, followed by qualitative and quantitative data analysis.
- **Findings**: Students use mobile devices, especially smartphones, for English study, finding them convenient and providing quick access to resources. They mainly use them for vocabulary learning and accessing online dictionaries and language apps. Most students use them in their leisure time for informal learning. Mobile device use was seen as a positive impetus for studying English more effectively.
- **Limitations**: Small sample size and homogeneity of participants from a single institution reduce the generalizability of the findings. The single semi-structured interview may 

### 3.3 Extract Paper Details & Metadata
#### Extract Metadata in JSON
- Showing how Gemini can return structured JSON

In [8]:
metadata_prompt = f"""
Extract the following structured data in JSON format:
- title
- authors
- year (if mentioned)
- keywords (if mentioned)

Paper:
{paper_text}
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=metadata_prompt
)

print("📋 Extracted Metadata:\n", response.text)


📋 Extracted Metadata:
 ```json
{
  "title": "A look at advanced learners’ use of mobile devices for English language study: Insights from interview data",
  "authors": [
    "Mariusz Kruk"
  ],
  "year": 2017,
  "keywords": [
    "Learner autonomy",
    "mobile devices",
    "advanced EFL learners",
    "learning English"
  ]
}
```


## 4. 🎯 Finding Related Papers with ArXiv API

### 4.1 Detect Research Topic
- Used to query arXiv for relevant papers

### 4.2. Retrieve and Embed Related Papers
- Fetch 10 results using arXiv API

- Embed using sentence-transformers

- Index and search using FAISS

- Return top 5 semantically similar papers

### 4.3 Compare with Similar Papers
- Few-shot comparison prompt to Gemini

- Output: structured comparison (title, similarities, differences, relation)

#### 4.1 Gemini Extracts Topic (Short Phrase)

In [9]:
# Gemini extracts the research topic from the research paper text the student gives
prompt = f"""
From the following text, extract the main research topic. Just return a clear short phrase:

{paper_text}
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)

detected_topic = response.text.strip()
print("🎯 Detected Research Topic:", detected_topic)


🎯 Detected Research Topic: Advanced learners' use of mobile devices for English language study.


In [10]:
import requests
import xml.etree.ElementTree as ET

def search_arxiv_papers(query, max_results=5):
    query = query.replace(" ", "+")
    url = f"http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={max_results}"
    response = requests.get(url)
    root = ET.fromstring(response.content)

    results = []
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        title = entry.find('{http://www.w3.org/2005/Atom}title').text.strip()
        summary = entry.find('{http://www.w3.org/2005/Atom}summary').text.strip()
        link = entry.find('{http://www.w3.org/2005/Atom}id').text.strip()
        authors = [author.find('{http://www.w3.org/2005/Atom}name').text for author in entry.findall('{http://www.w3.org/2005/Atom}author')]
        results.append({
            'title': title,
            'summary': summary,
            'authors': authors,
            'link': link
        })

    return results


#### 4.2 Passing The Detected Topic to RAG Flow and Getting the Top-5 Related Papers, Their Summaries and Their Links

In [11]:
# 1. Getting the research papers from arXiv
papers = search_arxiv_papers(detected_topic, max_results=10)

embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Embedding papers' summaries
summaries = [p["summary"] for p in papers]
summary_embeddings = embed_model.encode(summaries)

# 3. Embedding user query (PDF or typed topic)
query_embedding = embed_model.encode([detected_topic])

# 4. Search via FAISS
dimension = summary_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(summary_embeddings)
D, I = index.search(query_embedding, k=5)

# 5. Displaying the top k related paper matches
top_papers = []
print("📚 Top Related Papers:")
for idx in I[0]:
    p = papers[idx]
    top_papers.append(p)
    print(f"\n📄 Title: {p['title']}")
    print("Authors:", ", ".join(p["authors"]))
    print("Summary:", p["summary"][:1000], "...")
    print("🔗 Link:", p["link"])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

📚 Top Related Papers:

📄 Title: A Theoretical Approach to initiate Mobile Assisted Language Learning
  among school leavers and University Students of Sri Lanka
Authors: Fazeena Jamaldeen, Yamaya Ekanayaka, Kamalanath Hewagamage
Summary: Mobile devices have proven to be an appropriate tool in the area of mobile
language learning by expanding the horizons of learning beyond the classroom.
The power of mobile devices and advanced technologies have created ease of
access to educational resources and material to learners. This study is a part
of an ongoing research focusing on enhancing English language among school
leavers and undergraduates. The objective of this paper is to present the
results of a study conducted to identify the prospects of m-learning among the
target study community, who were school leavers and undergraduates of Sri
Lanka. A design based research methodology will be used in this study along
with activity theory as a framework in order to analyse the situation and fin

#### 4.3 Gemini Cross-Paper Analysis
This part summarizes the trends across top matched papers and compares them with the student's sample research paper 

In [12]:
# FEW-SHOT + STRUCTURED COMPARISON PROMPT
few_shot_examples = """
You are an academic research assistant. You will compare a user's research paper with other papers. 
For each comparison, output:

- Title of the related paper
- Similarities with the user's paper
- Differences or unique contributions
- Is it complementary, overlapping, or different in focus?

Example:

User Paper Summary:
"This paper explores how AI tutors can enhance student engagement in online learning platforms. The study uses 30 undergraduate students and measures voluntary interactions."

Related Paper Summary:
"This study examines virtual agents in MOOCs and how their interaction levels correlate with learning outcomes. It includes a sample of 200 postgraduate learners."

Comparison:
- Title: "Virtual Agents and Learning Outcomes in MOOCs"
- Similarities:
  - Both explore AI agents in educational settings
  - Both measure engagement-related metrics
- Differences:
  - User's paper focuses on undergrads; related paper uses postgrads
  - Related paper uses MOOCs; user’s paper is a blended course
- Relation: Complementary — same area, different cohorts and platforms
"""

# Build combined prompt with user paper + related 5
combined_input = f"""
{few_shot_examples}

Now compare the following user paper with the five related papers.

User Paper Summary:
{summary}
"""

for i, p in enumerate(top_papers):
    combined_input += f"""

Related Paper {i+1} Summary:
Title: {p["title"]}
Authors: {", ".join(p["authors"])}
Link: {p["link"]}
Summary: {p["summary"]}
"""

# Ask Gemini to perform comparison
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=combined_input
)

# OUTPUT — Only structured comparison with paper numbers + links
print(" USER'S PAPER SUMMARY:\n")
print(summary)

print("\n COMPARISON RESULTS:\n")

# Optional: Add paper numbers + links before displaying full comparison
for i, p in enumerate(top_papers):
    print(f"{i+1}. 📄 {p['title']}")
    print(f"   🔗 {p['link']}\n")

print(" Gemini's Structured Comparison Output:\n")
print(response.text)


 USER'S PAPER SUMMARY:

- **Title**: A look at advanced learners’ use of mobile devices for English language study: Insights from interview data
- **Focus**: How advanced English learners use mobile devices for language study and how it meets their needs and goals
- **Methodology**: Semi-structured interviews with 20 Polish university students of English philology, followed by qualitative and quantitative data analysis.
- **Findings**: Students use mobile devices, especially smartphones, for English study, finding them convenient and providing quick access to resources. They mainly use them for vocabulary learning and accessing online dictionaries and language apps. Most students use them in their leisure time for informal learning. Mobile device use was seen as a positive impetus for studying English more effectively.
- **Limitations**: Small sample size and homogeneity of participants from a single institution reduce the generalizability of the findings. The single semi-structured in

## 5. 📚  Generate Abstract & Outline 
- Based on Gemini summary, generate an academic outline

In [15]:
outline_prompt = f"""
You are a research assistant. Based on this summary, write an academic abstract and a proposed outline.

Summary:
{summary}
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=outline_prompt
)

print(" Generated Abstract & Outline:\n", response.text)


 Generated Abstract & Outline:
 Okay, here's an academic abstract and a proposed outline based on the summary you provided.

**Academic Abstract**

This study investigates the use of mobile devices by advanced English language learners to support their language acquisition.  Semi-structured interviews were conducted with 20 Polish university students enrolled in English philology programs to explore their practices, perceptions, and the perceived benefits of mobile-assisted language learning (MALL).  Qualitative and quantitative analysis of the interview data revealed that smartphones are a frequently utilized tool, primarily for vocabulary acquisition, accessing online dictionaries, and leveraging language learning applications.  Learners reported convenience and accessibility as key advantages, often incorporating mobile devices into their leisure time for informal language learning. The findings suggest a positive correlation between mobile device use and perceived gains in English 

## 6. 🔍 Evaluate Student Writing -
#### Now the student adds her/his new thoughts and sentences that she/he wants to add. (Gen AI Evaluation)
- Using Gemini to evaluate academic tone, grammar, clarity


In [16]:
# Defining the paragraph that the students wants to evaluate
student_paragraph = """
AI makes education better by helping teachers and students. It gives feedback and makes studying easier.
"""

# Creating the few-shot prompt using the paragraph
eval_prompt = f"""
You are a university professor evaluating student writing. 
Give detailed feedback on the following areas:
- clarity
- grammar
- academic tone
- suggestions for improvement

Here are two examples:

Example 1:
Paragraph: "AI is very good and helpful. It can do a lot of stuff and make things easier."
Feedback:
- Clarity: The paragraph is vague and general.
- Grammar: Grammatically simple but technically correct.
- Academic tone: Too informal for academic writing. Avoid phrases like "a lot of stuff."
- Suggestions: Specify what tasks AI helps with. Use precise language like "artificial intelligence facilitates tasks such as data analysis and personalization."

Example 2:
Paragraph: "The internet is a thing that many people use every day and it helps in learning and doing things faster."
Feedback:
- Clarity: The message is understandable but lacks specificity.
- Grammar: Slightly awkward phrasing ("a thing that many people use").
- Academic tone: Informal; better to use "a widely-used technological tool."
- Suggestions: Replace vague phrases with specific examples of how the internet aids learning. Aim for more academic phrasing.

Now evaluate this paragraph:

Paragraph: "{student_paragraph}"
Feedback:
"""

# Sending the prompt to Gemini
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=eval_prompt
)

# Displaying the feedback
print("Gemini Feedback:\n")
print(response.text)



Gemini Feedback:

Okay, here's my detailed feedback on the provided paragraph, broken down into the requested categories:

**Paragraph:** "AI makes education better by helping teachers and students. It gives feedback and makes studying easier."

**Feedback:**

*   **Clarity:** The paragraph conveys a general idea, but lacks specific details and examples. It's unclear how AI helps teachers, how it gives feedback, and in what ways it makes studying easier. The claim that "AI makes education better" is a broad statement that needs further support and qualification. What aspect of education is better? For which students or teachers?

*   **Grammar:** The grammar is technically correct and uses simple sentence structures. However, the simplicity contributes to the lack of depth and sophistication expected in academic writing. The repetition of "It" can also be slightly awkward and could be varied.

*   **Academic tone:** The tone is too simplistic and informal for academic writing. Phrases 

## 🧾 Conclusion

In this project, we demonstrated a comprehensive and practical use of Generative AI to support academic research tasks. By combining document understanding, few-shot prompting, vector-based retrieval, and structured output generation, we built an intelligent assistant that can:

- Read and summarize complex research papers,or directly work with the students' research papers which are in progress

- Identify their core topics,

- Retrieve and compare similar papers from arXiv,

- Evaluate academic writing and suggest improvements,

- Generate structured abstracts and outlines for further work.

This solution is especially valuable for students, researchers, and educators navigating large volumes of academic content. It helps accelerate literature reviews, support critical thinking, and encourage academic writing best practices — all through a single intelligent interface.

**The system leverages **6 GenAI** capabilities**, showing the versatility and depth of modern LLMs when paired with retrieval and evaluation components. With further development, this could evolve into a real-time agent-based research assistant or a web application, making research more accessible, personalized, and efficient.

